In [ ]:
import obi_auth

from obi_notebook.get_projects import get_projects
from obi_notebook.get_entities import get_entities

from entitysdk.models import CellMorphology, EMCellMesh, EMDenseReconstructionDataset, Circuit
from entitysdk import Client
from entitysdk._server_schemas import AssetLabel

token = obi_auth.get_token(environment="staging", auth_mode="daf")
project_context = get_projects(token, env="staging")

In [ ]:
entity_client = Client(token_manager=token, project_context=project_context, environment="staging")
morphology_ids = []

get_entities(entity_type="cell-morphology", token=token,
             project_context=project_context, env="staging",
             result=morphology_ids)

In [ ]:
from obi_one.scientific.tasks.em_synapse_mapping import EMSynapseMappingTask, EMSynapseMappingSingleConfig
from obi_one.scientific.from_id import cell_morphology_from_id

m = cell_morphology_from_id.CellMorphologyFromID(id_str=morphology_ids[0])
task_config = EMSynapseMappingSingleConfig(initialize=EMSynapseMappingSingleConfig.Initialize(
    spiny_neuron=m,
), coordinate_output_root="obi-one-test")

task = EMSynapseMappingTask(config=task_config)

In [ ]:
pt_root_id = int(m.entity(entity_client).name.split("-")[-1])
found_circuits = entity_client.search_entity(entity_type=Circuit, query={
    "name__ilike": f"{pt_root_id}"
}, limit=10).all()
if len(found_circuits) > 0:
    print(f"Found the following already existing entities that may built from the selected morphology with identifier {pt_root_id}:")
    print([_circ.name for _circ in found_circuits])
    print("Are you sure you want to continue?")
else:
    task.execute(entity_client)

# OLD, MANUAL VERSION BELOW

In [ ]:
import os 
from pathlib import Path
from neurom.io.utils import load_morphology_with_spines

db_client = entity_client
out_root = Path("obi-one-test")
os.makedirs(out_root / "morphologies/morphology", exist_ok=True)
morph_entity = m.entity(db_client)

fn_morphology_out_h5 = os.path.join("morphologies", morph_entity.name + ".h5")
fn_morphology_out_swc = os.path.join("morphologies/morphology", morph_entity.name + ".swc")
m.write_spiny_neuron_h5(out_root / fn_morphology_out_h5, db_client=db_client)
smooth_morph = m.neurom_morphology(db_client)
smooth_morph.to_morphio().as_mutable().write(out_root / fn_morphology_out_swc)
spiny_morph = load_morphology_with_spines(str(out_root / fn_morphology_out_h5))



In [ ]:
morphology_entity = entity_client.get_entity(entity_id=morphology_ids[0], entity_type=CellMorphology,
                                      project_context=project_context)
pt_root_id = int(morphology_entity.name.split("-")[1])
source_mesh_entity = list(entity_client.search_entity(entity_type=EMCellMesh, query={
    "dense_reconstruction_cell_id": pt_root_id
}))[0]
source_dataset = entity_client.get_entity(entity_id=source_mesh_entity.em_dense_reconstruction_dataset.id,
                                   entity_type=EMDenseReconstructionDataset)
spiny_morph_assets = [asset for asset in morphology_entity.assets if asset.label == AssetLabel.morphology_with_spines]

if len(spiny_morph_assets) == 0:
    print("This morphology has no representation with spines. Please select a morphology with spines!")

fn = entity_client.download_file(entity_id=morphology_entity.id, entity_type=CellMorphology, asset_id=spiny_morph_assets[0].id,
                          project_context=project_context, output_path=morphology_entity.name+".h5")

In [ ]:
found_circuits = entity_client.search_entity(entity_type=Circuit, query={
    "name__ilike": f"{pt_root_id}"
}, limit=10).all()
if len(found_circuits) > 0:
    print(f"Found the following already existing entities that may built from the selected morphology with identifier {pt_root_id}:")
    print([_circ.name for _circ in found_circuits])
    print("Are you sure you want to continue?")

In [ ]:
import os, shutil
import numpy

from obi_one.scientific.library.map_em_synapses import map_afferents_to_spiny_morphology, write_edges, write_nodes
from obi_one.scientific.library.map_em_synapses.map_synapse_locations import synapse_info_df
from obi_one.scientific.library.map_em_synapses.write_sonata_edge_file import _STR_PRE_NODE, _STR_POST_NODE
from obi_one.scientific.library.map_em_synapses.write_sonata_nodes_file import assemble_collection_from_specs

from neurom import load_morphology_with_spines


# START: USER CONFIGURATION
#  SONATA
out_root = "./output/sonata"
edge_population_name = "synaptome_afferents"
node_population_pre = "synaptome_afferent_neurons"
node_population_post = "biophysical_neuron"

# OUTPUTS
fn_edges_out = f"{pt_root_id}-sonata_edges.h5"
fn_nodes_out = f"{pt_root_id}-sonata_nodes.h5"
# END: USER CONFIGURATION

# MORE OUTPUTS
fn_morphology_out_h5 = os.path.join("morphologies", morphology_entity.name + ".h5")
fn_morphology_out_swc = os.path.join("morphologies/morphology", morphology_entity.name + ".swc")

# FROM PROVENANCE
#  CAVECLIENT
_version = source_mesh_entity.release_version
_datastack_name = source_dataset.cave_datastack
_cave_client_url = source_dataset.cave_client_url

# LOAD DATA
m = load_morphology_with_spines(fn, spines_are_centered=True)
os.makedirs(os.path.join(out_root, "morphologies/morphology"), exist_ok=True)
shutil.copy(fn, os.path.join(out_root, "morphologies"))
morphio_morph = m.to_morphio().as_mutable()
morphio_morph.soma.diameters += 1E-3
morphio_morph.write(os.path.join(out_root, fn_morphology_out_swc))


## Load afferent synapses using CAVEclient

In [ ]:
from caveclient import CAVEclient

client = CAVEclient(_datastack_name, server_address=_cave_client_url)
client.version = _version
info = client._info.get_datastack_info()

syns = synapse_info_df(client, pt_root_id, client.info.viewer_resolution(),
                       col_location="post_pt_position")
print(f"Loaded {len(syns)} synapses.")

## Find locations of synapses on the morphology

In [ ]:
mapped_synapses_df, mesh_res = map_afferents_to_spiny_morphology(m, syns, add_quality_info=True)

In [ ]:
def plot_mapping_stats(mapped_synapses_df, mesh_res,
                       plt_max_dist=3.0, nbins=99):
    from matplotlib import pyplot as plt

    dbins = numpy.linspace(0, plt_max_dist, nbins)
    w = numpy.mean(numpy.diff(dbins))

    frst_dist = numpy.maximum(mapped_synapses_df["distance"], 0.0)
    sec_dist = mapped_synapses_df["competing_distance"]

    fig = plt.figure(figsize=(2.5, 4))
    ax = fig.add_subplot(2, 1, 1)

    ax.bar(dbins[1:], numpy.histogram(frst_dist, bins=dbins)[0], width=w, label="Dist.: Nearest structure")
    ax.bar(dbins[1:], numpy.histogram(sec_dist, bins=dbins)[0], width=w, label="Dist.: Second nearest structure")
    ymx = ax.get_ylim()[1] * 0.85
    ax.plot([mesh_res, mesh_res], [0, ymx], color="black", label="Mesh resolution")
    ax.set_ylabel("Synapse count")
    ax.set_frame_on(False)
    plt.legend()

    ax = fig.add_subplot(2, 1, 2)

    ax.bar(dbins[1:], numpy.histogram(sec_dist - frst_dist, bins=dbins)[0],
           facecolor="grey", width=w, label="Difference")
    ymx = ax.get_ylim()[1] * 0.85
    ax.plot([mesh_res, mesh_res], [0, ymx], color="black", label="Mesh resolution")
    ax.set_xlabel("um")
    ax.set_ylabel("Synapse count")
    ax.set_frame_on(False)
    plt.legend()
    print(((sec_dist - frst_dist) >= mesh_res).mean())

    return fig

plot_mapping_stats(mapped_synapses_df, mesh_res)

## Write mapped synapses to SONATA edges

In [ ]:
pre_pt_root_to_sonata = syns["pre_pt_root_id"].drop_duplicates().reset_index(drop=True).reset_index().set_index("pre_pt_root_id")
post_pt_root_to_sonata = syns["post_pt_root_id"].drop_duplicates().reset_index(drop=True).reset_index().set_index("post_pt_root_id")

syn_pre_post_df = pre_pt_root_to_sonata.loc[syns["pre_pt_root_id"]].rename(columns={"index": _STR_PRE_NODE})
syn_pre_post_df[_STR_POST_NODE] = 0
syn_pre_post_df = syn_pre_post_df.reset_index(drop=True)

write_edges(os.path.join(out_root, fn_edges_out), edge_population_name, syn_pre_post_df,
            mapped_synapses_df, node_population_pre, node_population_post)

## Load neuron info using CAVEclient
### Specify which information is needed and how it's formatted

In [ ]:
specs = {
    "synapse_class": {
        "table": 'aibs_metamodel_mtypes_v661_v2',
        "column": 'classification_system',
        "default": "extrinsic_neuron"
    },
    "cell_type": {
        "table": 'aibs_metamodel_mtypes_v661_v2',
        "column": 'cell_type',
        "default": "extrinsic_neuron"
    },
    "volume": {
        "table": 'aibs_metamodel_mtypes_v661_v2',
        "column": 'volume',
        "default": -1
    },
    "status_axon": {
        "table": 'proofreading_status_and_strategy',
        "column": 'status_axon',
        "default": "unknown"
    },
    "status_dendrite": {
        "table": 'proofreading_status_and_strategy',
        "column": 'status_dendrite',
        "default": "unknown"
    },
    "__position": {
        "table": 'aibs_metamodel_mtypes_v661_v2',
        "column": "pt_position",
        "resolution": {
            "x": info["viewer_resolution_x"] * 1E-3,
            "y": info["viewer_resolution_y"] * 1E-3,
            "z": info["viewer_resolution_z"] * 1E-3
        }
    }
}

### Assemble specified neuron information and write to nodes file

In [ ]:
coll_pre = assemble_collection_from_specs(client, specs, pre_pt_root_to_sonata)
coll_post = assemble_collection_from_specs(client, specs, post_pt_root_to_sonata)

coll_post.properties["morphology"] = f"morphology/{m.name}"
write_nodes(os.path.join(out_root, fn_nodes_out), node_population_pre, coll_pre, write_mode="w")
write_nodes(os.path.join(out_root, fn_nodes_out), node_population_post, coll_post, write_mode="a")

In [ ]:
sonata_cfg = {
  "components": {
    "biophysical_neuron_models_dir": "",
    "mechanisms_dir": "",
    "morphologies_dir": "",
    "point_neuron_models_dir": "",
    "synaptic_models_dir": "",
    "templates_dir": ""
  },
  "networks": {
    "edges": [
      {
        "edges_file": "$BASE_DIR/" + fn_edges_out,
        "populations": {
          edge_population_name: {
            "type": "chemical"
          }
        }
      }
    ],
    "nodes": [
      {
        "nodes_file": "$BASE_DIR/" + fn_nodes_out,
        "populations": {
          node_population_post: {
            "alternate_morphologies": {
              "h5v1": "$BASE_DIR/" + fn_morphology_out_h5
            },
            "biophysical_neuron_models_dir": "$BASE_DIR/emodels_hoc",
            "morphologies_dir": "$BASE_DIR/morphologies",
            "type": "biophysical"
          }
        }
      },
      {
        "nodes_file": "$BASE_DIR/" + fn_nodes_out,
        "populations": {
          node_population_pre: {
            "type": "virtual"
          }
        }
      }
    ]
  },
  "node_sets_file": "$BASE_DIR/node_sets.json",
  "version": 2.3,
  "manifest": {
    "$BASE_DIR": "./"
  }
}
with open(os.path.join(out_root, "circuit_config.json"), "w") as fid:
    import json
    json.dump(sonata_cfg, fid, indent=2)

## Compress circuit

In [ ]:
import subprocess

with open("output/sonata.tar", "wb") as fid:
    fid.write(
        subprocess.check_output([
            "tar",
            "-c",
            "output/sonata"
            ]
        )
    )
subprocess.check_call([
    "gzip",
     "-1",
    "output/sonata.tar"
])

# Register new Circuit entity and upload assets

In [ ]:
from entitysdk.models import Circuit
from entitysdk._server_schemas import CircuitBuildCategory, CircuitScale

circ_entity = Circuit(
    name=f"Afferent-synaptome-{pt_root_id}",
    description=f"Morphology skeleton with isolated spines and afferent synapses (Synaptome) of the neuron with pt_root_id {pt_root_id} in dataset {source_dataset.name}",
    number_neurons=1,
    number_synapses=len(mapped_synapses_df),
    number_connections=len(syn_pre_post_df["pre_node_id"].drop_duplicates()),
    scale=CircuitScale.single,
    build_category=CircuitBuildCategory.em_reconstruction,
    subject=source_dataset.subject,
    has_morphologies=True,
    has_electrical_cell_models=False,
    has_spines=True,
    brain_region=source_dataset.brain_region,
    experiment_date=source_dataset.experiment_date
)
existing_circuit = entity_client.register_entity(circ_entity,
                                          project_context=project_context)

In [ ]:
entity_client.upload_directory(
    entity_id=existing_circuit.id,
    entity_type=Circuit,
    name="sonata_synaptome",
    paths={
        "circuit_config.json": os.path.join(out_root, "circuit_config.json"),
        fn_nodes_out : os.path.join(out_root, fn_nodes_out),
        fn_edges_out: os.path.join(out_root, fn_edges_out),
        fn_morphology_out_h5: os.path.join(out_root, fn_morphology_out_h5),
        fn_morphology_out_swc: os.path.join(out_root, fn_morphology_out_swc)
        },
    label=AssetLabel.sonata_circuit,
    project_context=project_context
)

In [ ]:
from entitysdk._server_schemas import ContentType, AssetLabel

entity_client.upload_file(entity_id=existing_circuit.id,
                   entity_type=Circuit,
                   file_path="output/sonata.tar.gz",
                   file_content_type=ContentType.application_gzip,
                   asset_label=AssetLabel.compressed_sonata_circuit)

In [ ]:
entity_client.get_entity(existing_circuit.id, entity_type=Circuit).assets